# 텍스트 분류

In [1]:
from sklearn.datasets import fetch_20newsgroups

news_data=fetch_20newsgroups(subset='all', random_state=156)

In [2]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [3]:
import pandas as pd

print('target 클래스의 값과 분포도 \n', pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n', news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [5]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

In [6]:
train_news=fetch_20newsgroups(subset='train', remove=('headers','footers','quotes'), random_state=156)

x_train=train_news.data
y_train=train_news.target

test_news=fetch_20newsgroups(subset='test', remove=('headers','footers','quotes'), random_state=156)

x_test=test_news.data
y_test=test_news.target

print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data), len(test_news.data)))

학습 데이터 크기 11314, 테스트 데이터 크기 7532


### 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

cnt_vect=CountVectorizer()
cnt_vect.fit(x_train, y_train)

x_train_cnt_vect=cnt_vect.transform(x_train)
x_test_cnt_vect=cnt_vect.transform(x_test)

print('학습 데이터 텍스트의 CountVectorizer Shape : ',x_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape :  (11314, 101631)


학습 데이터로 피처를 추출한 결과, 11314개의 문서에서 101631 개의 단어가 있음을 볼 수 있다.

해당 피처를 사용하여 로지스틱 회귀를 적용해 뉴스그룹에 대한 분류를 예측해보자

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr_clf=LogisticRegression()
lr_clf.fit(x_train_cnt_vect, y_train)
pred=lr_clf.predict(x_test_cnt_vect)

print('CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

CountVectorized Logistic Regression의 예측 정확도는 0.605


C:\Users\river\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


TF-IDF 기반으로 데이터 변환

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect=TfidfVectorizer()
tfidf_vect.fit(x_train)

x_train_tfidf_vect=tfidf_vect.transform(x_train)
x_test_tfidf_vect=tfidf_vect.transform(x_test)

In [11]:
lr_clf=LogisticRegression()
lr_clf.fit(x_train_tfidf_vect, y_train)

pred=lr_clf.predict(x_test_tfidf_vect)

print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.674


TF-IDF가 단순 카운트 기반보다 훨씬 높은 예측 정확도를 제공한다. 일반적으로 문서 내에 텍스트가 많고
많은 문서를 가지는 텍스트분석에서 카운트 벡터화보다는 TF-IDF 벡터화가 좋은 예측 결과를 도출한다.

---

텍스트 분서게서 머신러닝 모델의 성능을 향상시키는 중요한 2가지 방법은 최적의 ML 알고리즘을 선택하는 것과
최상의 피처 전처리를 수행하는 것이다.

텍스트 정규화나 Count/TF-IDF 기반 피처 벡터화를 어떻게 효과적으로 적용했는지가
텍스트 기반의 머신러닝 성능에 큰 영향을 미칠 수 있다.

In [14]:
tfidf_vect=TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)
tfidf_vect.fit(x_train)

x_train_tfidf_vect=tfidf_vect.transform(x_train)
x_test_tfidf_vect=tfidf_vect.transform(x_test)

In [15]:
lr_clf=LogisticRegression()
lr_clf.fit(x_train_tfidf_vect, y_train)

pred=lr_clf.predict(x_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Vectorized Logistic Regression 의 예측 정확도는 0.692


GridSearchCV를 이용한 로지스틱 회귀의 하이퍼 파라미터 최적화 수행

In [16]:
from sklearn.model_selection import GridSearchCV

params={'C':[0.01, 0.1, 1, 5, 10]}
grid_cv_lr=GridSearchCV(lr_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_lr.fit(x_train_tfidf_vect, y_train)

print('Logistic Regression best C parameter : ', grid_cv_lr.best_params_)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


C:\Users\river\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\river\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

KeyboardInterrupt: 

In [ ]:
pred=grid_cv_lr.predict(x_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

파이프라인 생성

In [ ]:
from sklearn.pipeline import Pipeline

pipeline=Pipeline([('tfidf_vect',TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)),
                  ('lr_clf',LogisticRegression(C=10))])

In [ ]:
pipeline.fit(x_train, y_train)
pred=pipeline.predict(x_test)

print('Pipeline을 통한 Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

GridSearchCV 와 Pipeline 의 결합

In [ ]:
params={'tfidf_vect__ngram_range':[(1,1),(1,2),(1,3)],
       'tfidf_vect__max_df':[100,300,700],
       'lr_clf__C':[1,5,10]}

In [ ]:
grid_cv_pipe=GridSearchCV(pipeline, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_pipe.fit(x_train, y_train)

print(grid_cv_pipe.best_params, grid_cv_pipe.best_score_)

pred=grid_cv_pipe.predict(x_test)

print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))